In [ ]:
""""0. initialize libraries and setup tools"""

%load_ext jupyternotify

import numpy as np
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import models
from torchsummary import summary

env = gym.make('MountainCar-v0')

# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()  # checks if one is running IPhyton environment like jupyter notebook
if is_ipython:
    from IPython import display

plt.ion()  # interactive mode on, allows automatic plots when data is updated (whithout calling plt.show every time)

# if GPU is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
   
    
    
    
""" 1.1. Helper functions """

def plot_durations(durations, hyperparameters):
    plt.figure()
    durations_t = torch.tensor(durations, dtype=torch.float)
    plt.title('Duration per Episode')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    if len(durations_t) >= 100:
        means = durations_t[100:].unfold(0, 100, 1).mean(1).view(-1)
        plt.plot(range(100, len(durations)-99), means.numpy(), label='100 Moving Average')
    plt.savefig(f"DQN_Duration_Hyperparameters_{hyperparameters}.png", dpi=300)
    plt.close()
    
def plot_curve(data, title, xlabel, ylabel, savefig=False):
    plt.plot(data, label = 'title')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if savefig: 
        plt.savefig(f"{title}.png", dpi=300) # Use f-string to concatenate strings and variable
        
def plot_scatter(data, xlabel, ylabel):
    data_t = torch.tensor(data, dtype=torch.float)
    plt.title('Result')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.scatter(list(range(len(data))), data_t.numpy()) # usually for plt.scatter you need (x,y) as arguments

def plot_with_average(data, xlabel, ylabel):
    plt.figure(1)
    data_t = torch.tensor(data, dtype=torch.float)
    plt.title('Result')
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.plot(list(range(len(data))), data_t.numpy(), label='Cumulative Reward Per Episode') 
    # Take 100 episode averages and plot them too
    if len(data_t) > 100:  # Check if data has more than 100 episodes
        # Calculate moving average starting from episode 100
        means = data_t[100:].unfold(0, 100, 1).mean(1).view(-1)
        # Plot the average line starting from episode 100
        plt.plot(range(100, len(data)-99), means.numpy(), label='Moving Average (100 episodes)')

def cumulative_sum(input_list):
    result = []
    running_total = 0
    for element in input_list:
        running_total += element
        result.append(running_total)
    return result



    
""""1.2. Define ReplayBuffer and deep q network"""

Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayBuffer(object):

    def __init__(self, capacity):
        self.buffer = deque([], maxlen=capacity)

    def push(self, *args):
        self.buffer.append(Transition(*args)) #Save a transition

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def __len__(self):
        return len(self.buffer)

class DQN(nn.Module):

    def __init__(self, n_observations, n_actions, n_nodes_per_layer=64, n_layers=2):
        super(DQN, self).__init__()
        self.layers = nn.ModuleList([nn.Linear(n_observations, n_nodes_per_layer)])
        for _ in range(n_layers - 1):
            self.layers.append(nn.Linear(n_nodes_per_layer, n_nodes_per_layer))
        self.output_layer = nn.Linear(n_nodes_per_layer, n_actions)

    def forward(self, x):
        for layer in self.layers:
            x = F.relu(layer(x))
        return self.output_layer(x)
            
        
    

"""" 2. Agent class """

class DQNAgent:
    def __init__(self, BATCH_SIZE, GAMMA, EPS_START, EPS_END, EPS_DECAY, TAU, LR, n_actions, n_observations):
        self.BATCH_SIZE = BATCH_SIZE
        self.GAMMA = GAMMA
        self.EPS_START = EPS_START
        self.EPS_END = EPS_END
        self.EPS_DECAY = EPS_DECAY
        self.TAU = TAU
        self.LR = LR
        self.n_actions = n_actions
        self.n_observations = n_observations
        self.memory = ReplayBuffer(10000)
        self.policy_net = DQN(n_observations, n_actions).to(device)
        self.target_net = DQN(n_observations, n_actions).to(device)
        self.optimizer = optim.Adam(self.policy_net.parameters(), lr=LR)
        self.steps_done = 0
        self.episode_durations = []
        self.loss_per_episode = []
        self.cumulative_reward_per_episode = []
        self.cumulative_environment_reward_per_episode = []
        self.cumulative_auxiliary_reward_per_episode = []
        self.agent_performance = []


    def select_action(self, state):
        sample = random.random()
        eps_threshold = self.EPS_END + (self.EPS_START - self.EPS_END) * math.exp(-1. * self.steps_done / self.EPS_DECAY)
        self.steps_done += 1
        if sample > eps_threshold:
            with torch.no_grad():
                return self.policy_net(state).max(1)[1].view(1, 1)
        else:
            return torch.tensor([[random.randrange(self.n_actions)]], device=device, dtype=torch.long)

    def optimize_model(self):
        if len(self.memory) < self.BATCH_SIZE:
            return
        transitions = self.memory.sample(self.BATCH_SIZE)
        batch = Transition(*zip(*transitions))
        
        non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                                batch.next_state)), device=device, dtype=torch.bool)
        non_final_next_states = torch.cat([s for s in batch.next_state
                                           if s is not None])
        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        
        state_action_values = self.policy_net(state_batch).gather(1, action_batch)
        
        next_state_values = torch.zeros(self.BATCH_SIZE, device=device)
        with torch.no_grad():
            next_state_values[non_final_mask] = self.target_net(non_final_next_states).max(1).values
        
        expected_state_action_values = (next_state_values * self.GAMMA) + reward_batch
        expected_state_action_values = expected_state_action_values.float()

        loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))
        
        self.optimizer.zero_grad()
        loss.backward()
        for param in self.policy_net.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()
        
        return loss.item()

    def update(self, num_episodes):
        for episode in tqdm(range(num_episodes), desc="Episodes"):
            state, info = env.reset()
            state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
            cumulative_reward_t = 0
            environment_reward_t = 0
            auxiliary_reward_t = 0
            running_loss = []
            for t in count():
                action = self.select_action(state)
                observation, reward, terminated, truncated, _ = env.step(action.item())
                environment_reward_t += reward
                aux_reward = 0.1*observation[0]
                auxiliary_reward_t += aux_reward
                reward = reward + aux_reward
                cumulative_reward_t += reward
                reward = torch.tensor([reward], device=device)
                done = terminated or truncated
                
                if not done:
                    next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)
                    self.agent_performance.append(1)
                else:
                    next_state = None
                if truncated:
                    self.agent_performance.append(0)
                    
                self.memory.push(state, action, next_state, reward)
                state = next_state
                
                loss_value = self.optimize_model()
                
                target_net_state_dict = self.target_net.state_dict()
                policy_net_state_dict = self.policy_net.state_dict()
                for key in policy_net_state_dict:
                    target_net_state_dict[key] = policy_net_state_dict[key]*self.TAU + target_net_state_dict[key]*(1-self.TAU)
                self.target_net.load_state_dict(target_net_state_dict)

                if done:
                    self.episode_durations.append(t + 1)
                    break
                    
                running_loss.append(loss_value)
            
            filtered_running_loss = [loss for loss in running_loss if loss is not None]
            if filtered_running_loss:
                self.loss_per_episode.append(np.mean(filtered_running_loss))

            self.cumulative_reward_per_episode.append(cumulative_reward_t)
            self.cumulative_environment_reward_per_episode.append(environment_reward_t)
            self.cumulative_auxiliary_reward_per_episode.append(auxiliary_reward_t)
        
    

"""" 3. Run Model """

def train_agent_with_hyperparameters(BATCH_SIZE, GAMMA, EPS_START, EPS_END, EPS_DECAY, TAU, LR, num_episodes=3000):
    n_actions = env.action_space.n
    n_observations = env.observation_space.shape[0]
    agent = DQNAgent(BATCH_SIZE, GAMMA, EPS_START, EPS_END, EPS_DECAY, TAU, LR, n_actions, n_observations)
    agent.update(num_episodes)
    hyperparameters = f"BATCH_SIZE={BATCH_SIZE}_GAMMA={GAMMA}_EPS_START={EPS_START}_EPS_END={EPS_END}_EPS_DECAY={EPS_DECAY}_TAU={TAU}_LR={LR}"
    plot_durations(agent.episode_durations, hyperparameters)

def hyperparameter_tuning():
    BATCH_SIZE = 128
    GAMMA = 0.99
    EPS_START = 0.9
    EPS_END = 0.1
    EPS_DECAY = 100000
    TAU = 0.0005 
    LR = 1e-4

#     for BATCH_SIZE in [32, 64, 128]:
#         train_agent_with_hyperparameters(BATCH_SIZE, GAMMA, EPS_START, EPS_END, EPS_DECAY, TAU, LR)
    for GAMMA in [0.95, 0.90]:
        train_agent_with_hyperparameters(BATCH_SIZE, GAMMA, EPS_START, EPS_END, EPS_DECAY, TAU, LR)
#     for EPS_START in [1.0, 0.9, 0.8]:
#         train_agent_with_hyperparameters(BATCH_SIZE, GAMMA, EPS_START, EPS_END, EPS_DECAY, TAU, LR)
#     for EPS_END in [0.1, 0.05, 0.01]:
#         train_agent_with_hyperparameters(BATCH_SIZE, GAMMA, EPS_START, EPS_END, EPS_DECAY, TAU, LR)
#     for EPS_DECAY in [10000, 100000, 1000000]:
#         train_agent_with_hyperparameters(BATCH_SIZE, GAMMA, EPS_START, EPS_END, EPS_DECAY, TAU, LR)
#     for TAU in [0.00005, 0.0005, 0.005]:
#         train_agent_with_hyperparameters(BATCH_SIZE, GAMMA, EPS_START, EPS_END, EPS_DECAY, TAU, LR)
#     for LR in [0.001, 0.0005, 0.0001]:
#         train_agent_with_hyperparameters(BATCH_SIZE, GAMMA, EPS_START, EPS_END, EPS_DECAY, TAU, LR)




# Run the hyperparameter tuning
hyperparameter_tuning()

<IPython.core.display.Javascript object>

Episodes:   5%|█▉                                     | 150/3000 [00:44<14:42,  3.23it/s]

In [ ]:
#============ Duration per Episode =====================

# figure_title = 'DQN with position as auxiliary reward' # = filename
# plot_durations(figure_title, savefig=False, show_result=True)

In [ ]:
print(agent.episode_durations)

In [ ]:
#============ Episode Durations, Scatterplot =====================

# plot_scatter(episode_durations, 'Episode', 'Duration')
# plt.show()

In [ ]:
#============ Loss Curve =====================

plot_curve(agent.loss_per_episode, 'Loss Curve', 'Training Step', 'Loss')
plt.show()

# report loss
print("final loss:", loss_per_episode[-1])

# report average cumulative reward per ep
average_cumulative_reward_per_episode = np.mean(cumulative_reward_per_episode)
print('average_cumulative_reward_per_episode = ', average_cumulative_reward_per_episode)


In [ ]:
#============ Cumulative Reward per Episode =====================

#plot_with_average(cumulative_reward_per_episode, 'Episode', 'Cumulative Reward')

In [ ]:
#============ Cumulative Reward per Episode, Scatterplot =====================

# plot_scatter(cumulative_reward_per_episode, 'Episode', 'Cumulative Reward')
# plt.show()

In [ ]:
#============ Composition of Averaged Cumulative Reward per Episode =====================

data_t = torch.tensor(cumulative_reward_per_episode, dtype=torch.float)
plt.title('Composition of Averaged Cumulative Reward per Episode')
plt.xlabel('Episode')
plt.ylabel('Cumulative Reward')
c_e_r_p_e_t = torch.tensor(cumulative_environment_reward_per_episode, dtype=torch.float)
c_a_r_p_e_t = torch.tensor(cumulative_auxiliary_reward_per_episode, dtype=torch.float)
if len(c_e_r_p_e_t) > 100:  # Check if data has more than 100 episodes
        means = data_t[100:].unfold(0, 100, 1).mean(1).view(-1)
        # Plot the average line starting from episode 100
        plt.plot(range(100, len(cumulative_reward_per_episode)-99), means.numpy(), label='Cumulative Reward Per Episode')
        # Calculate moving average starting from episode 100
        means = c_e_r_p_e_t[100:].unfold(0, 100, 1).mean(1).view(-1)
        # Plot the average line starting from episode 100
        plt.plot(range(100, len(cumulative_environment_reward_per_episode)-99), means.numpy(), label='Cumulative Environment Reward Per Episode')
        means = c_a_r_p_e_t[100:].unfold(0, 100, 1).mean(1).view(-1)
        # Plot the average line starting from episode 100
        plt.plot(range(100, len(cumulative_auxiliary_reward_per_episode)-99), means.numpy(), label='Cumulative Auxiliary Reward Per Episode')

plt.legend()
plt.show()

In [ ]:
# #============ Cumulative number of successes per Episode =====================

# cumulative_reward = cumulative_sum(cumulative_reward_per_episode)
# cumulative_environment_reward = cumulative_sum(cumulative_environment_reward_per_episode)
# cumulative_auxiliary_reward = cumulative_sum(cumulative_auxiliary_reward_per_episode)

# #separately depict the environment reward, the auxiliary reward and their sum
# #### SMOOTHEN IF NECESSARY
# plt.figure(1)
# plt.title('Composition of Cumulative Reward')
# plt.xlabel('Episode')
# plt.ylabel('Cumulative Reward')
# plt.plot(cumulative_reward, label='Cumulative Reward')
# plt.plot(cumulative_environment_reward, label='Environment Reward')
# plt.plot(cumulative_auxiliary_reward, label='Auxiliary Reward')

# plt.legend()  # Show legend

In [ ]:
#============ Cumulative number of successes =====================

cumulative_successes = cumulative_sum(agent_performance)
plot_curve(cumulative_successes, 'Cumulative Number of Successes', 'Episodes', 'Successes', savefig=False)